In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import pickle
import re
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import glob

from sklearn.metrics import accuracy_score,fbeta_score,classification_report,confusion_matrix
from sklearn.model_selection import train_test_split,KFold
from sklearn.svm import SVC
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

from nltk.corpus import stopwords
nltk.download('stopwords')
stop=stopwords.words("english")

from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
wl = WordNetLemmatizer()
ss = SnowballStemmer("english")
ps = PorterStemmer()

nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import string
email_words =["subject","cc","from","to"] 
def cleanMes(message):
    # remove other than a-z and removind doc symbols
    message = re.sub('\\\\[a-zA-Z0-9]*', ' ', message)
    message = re.sub('[^a-zA-Z]', ' ', message)
    # to lower case
    message = message.lower()
    # removing stop words
    message = message.split()
    #  for lematize use wl.lemmatize insted of ss.stem(), for portetstemmer use ps.stem()
    words =[ss.stem(word) for word in message if (word not in stop and word not in email_words and word[-1]!='$' and word[0]!='$' and not word.isnumeric())]
    return " ".join(words)



In [ ]:
# spam_words = []
# ham_words=[]

# def spamWords(spamMessages):
#     global spam_words 
#     words = [word for word in word_tokenize(spamMessages)]
#     spam_words = spam_words + words
    
# def hamWords(hamMessages):
#     global ham_words
#     words = [word for word in word_tokenize(hamMessages)]
#     ham_words = ham_words + words


In [ ]:
def encoding(e):
    if(e == "spam"):
        return 1
    else:
        return 0

     


In [ ]:
# load data
def loadData():
    data = pd.read_csv('drive/My Drive/prml assignment/assignment 3/eron1_spam.csv', sep='\t', names=["label", "message"])
    data.shape
    data["message"] = data["message"].apply(cleanMes)
    data["label"] = data["label"].apply(encoding)
    data = data.iloc[1:]
    return data


In [ ]:
def createCountVector(data):
    cv = CountVectorizer()
    X=cv.fit(data["message"])
    X = cv.fit_transform(data["message"]).toarray()
    # pickle.dump(cv.vocabulary_,open("drive/My Drive/prml assignment/assignment 3/cv.pkl","wb"))
    X = pd.DataFrame(X,columns=cv.get_feature_names())
    print(len(X))
    return X


In [ ]:
def SVM():
    # del data
    data = loadData()
    X = createCountVector(data)
    X, X_test, Y, Y_test = train_test_split(X, data["label"], test_size = 0.10, random_state = 10)
    # Y=data["label"]
    del data
    # print(Y_train.iloc[:5])
    svm = SVC(kernel='linear')
    
    # print(len(X_train))

    # for manual spliting 
    # d=1000
    # split = [i*d for i in range(len(X_train)//d)] + [len(X_train)%d]
    # print(split)
    # for i in range(len(split)-1):
    #     x_train, y_train = X_train.iloc[split[i]:split[i+1]] , Y_train.iloc[split[i]:split[i+1]] 
    #     scores.append(svm.fit(x_train, y_train).score(X_test,Y_test))

    # for Kfold split
    scores = []
    cv = KFold(n_splits=4)
    for train_index,test_index in cv.split(X):
        x_train,x_test, y_train,y_test = X.iloc[train_index],X.iloc[test_index] ,Y.iloc[train_index],Y.iloc[test_index]
        scores.append(svm.fit(x_train, y_train).score(x_test,y_test))
        # print(scores)
    return scores,svm,X_test,Y_test
   

In [ ]:

def plot(cm):
    plt.figure(figsize = (5, 5))
    plt.xlabel("x-axis")
    plt.ylabel("y-axis")
    plt.title("confusion matrix")
    sns.set(font_scale=1.5)
    sns.heatmap(cm,annot = True, fmt="n", xticklabels=['Not Spam', 'Spam'], yticklabels=['Not Spam', 'Spam'])
    # plt.savefig("hm",dpi=500)

In [ ]:
def train():
    a,b,x,y = SVM()
    print(a)
    # pickle.dump(b,open("drive/My Drive/prml assignment/assignment 3/SVM.sav","wb"))
    y_test = y
    svm=a
    # svm = pickle.load(open("drive/My Drive/prml assignment/assignment 3/svm.sav","rb"))
    y_pred = svm.predict(x)
    print(y_pred.shape,y_test.shape)
    print(accuracy_score(y_test,y_pred))
    # print(fbeta_score(y_test,y_pred,beta =0.5))
    print(classification_report(y_test,y_pred))
    cm=confusion_matrix(y_test,y_pred)
    plot(cm)


In [77]:
# download SVM modal and count vectorizer file
!gdown --id 1-6m8uaz-MU2pCmogjmKfJs0IqbXhkeK2
!gdown --id 1Yqd_yjRI9vQXcxiKr8F6L3U65iFG-X3x


Downloading...
From: https://drive.google.com/uc?id=1-6m8uaz-MU2pCmogjmKfJs0IqbXhkeK2
To: /content/SVM.sav
445MB [00:01, 259MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Yqd_yjRI9vQXcxiKr8F6L3U65iFG-X3x
To: /content/cv.pkl
100% 1.92M/1.92M [00:00<00:00, 123MB/s]


In [78]:


def testData(path):
    f=glob.glob(path)
    x_test = []
    for i in f:
        t = open(i,'r')
        x_test.append(t.read())
    # x_test.append("you have won prize lucky win beifit game play watch")
    # x_test.append("hi got it send by evening")
    # print(x_test)
    dict = {"message" : x_test}
    x_test = pd.DataFrame(dict)
    x_test["message"] = x_test["message"].apply(cleanMes)
    # print(x_test)

    
    cv = CountVectorizer(vocabulary=pickle.load(open("cv.pkl", "rb")))
    X = cv.transform(x_test["message"]).toarray()
    X = pd.DataFrame(X,columns=cv.get_feature_names())

    svm = pickle.load(open("SVM.sav","rb"))
    Y = svm.predict(X)
    # print(Y)
    dict = {"label" : Y}
    df = pd.DataFrame(dict)
    df.to_csv("output.csv")
    print(Y)



In [79]:
# put the path were test folder is present
path = "test/email*.txt"
testData(path)